In [1]:
%cd new_superpoint_transformer/

c:\Users\hany5\testing-app\new_superpoint_transformer


In [2]:
import numpy as np
from pyntcloud import PyntCloud
from src.data import Data
import torch

d:\Coding\envs\spt\lib\site-packages\pyntcloud\utils\numba.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_count(xyz, indices, out):
d:\Coding\envs\spt\lib\site-packages\pyntcloud\utils\numba.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_sum(xyz, indices, N, out):
d:\Coding\envs\spt\lib\site-packages\pyntcloud\utils\numba.py:19

In [3]:
ply_path = "C:\\Users\\hany5\\Downloads\\test\\_vswork_room15.ply"

In [4]:
cloud_file = PyntCloud.from_file(ply_path)

In [8]:
cloud_file.points['x'].dtype

dtype('float32')

In [48]:
pcd = PyntCloud.from_file(ply_path)

In [49]:
pcd

PyntCloud
1844000 points with 5 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: 9.349696159362793, -2.888927698135376, 1.6136233806610107
Other attributes:

In [4]:
def read_ply(file_path):
    """
    Read a .ply file and return a data object with the following fields:
    - 'points': (N, 3) numpy array with the point coordinates
    - 'colors': (N, 3) numpy array with the point colors
    """
    pcd = PyntCloud.from_file(file_path)
    
    points = pcd.points
    #Extract pos data
    pos = points[['x', 'y', 'z']].values

    #Extract color data
    colors = points[['red', 'green', 'blue']].values
    colors = colors / 255.0


    data = Data()

    data.pos = torch.tensor(pos, dtype=torch.float32)
    data.rgb = torch.tensor(colors, dtype=torch.float32)
    
    return data

In [5]:
from src.data import Data
from src.utils import init_config
from src.transforms import instantiate_datamodule_transforms

# we use init_config to load the configuration file and do the exact same preprocessing as in the training pipeline
cfg = init_config(overrides=[f"experiment=semantic/s3dis"])

transforms_dict = instantiate_datamodule_transforms(cfg.datamodule)
data = read_ply(ply_path)
nag = transforms_dict['pre_transform'](data)
from src.transforms import NAGRemoveKeys

nag = NAGRemoveKeys(level=0, keys=[k for k in nag[0].keys if k not in cfg.datamodule.point_load_keys])(nag)
nag = NAGRemoveKeys(level='1+', keys=[k for k in nag[1].keys if k not in cfg.datamodule.segment_load_keys])(nag)

# Move to device
nag = nag.cuda()

# Apply on-device transforms
nag = transforms_dict['on_device_test_transform'](nag)

tensor([[ 261.,  -81.,   92.],
        [ 275., -126.,    3.],
        [ 275., -126.,    3.],
        ...,
        [ 355., -149.,   30.],
        [ 356., -149.,   30.],
        [ 356., -149.,   30.]], device='cuda:0')


In [8]:
nag.show()

TypeError: save() missing 1 required positional argument: 'path'

In [39]:
import hydra

ckpt_path = "./checkpoints/last.ckpt"
# Instantiate the model and load pretrained weights
model = hydra.utils.instantiate(cfg.model)
model = model._load_from_checkpoint(ckpt_path)

# Set the model in inference mode on the same device as the input
model = model.eval().to(nag.device)

# Inference, returns a task-specific ouput object carrying predictions
with torch.no_grad():
    output = model(nag)

In [40]:
output.semantic_pred.shape, nag.num_points

(torch.Size([1749]), [64177, 1749, 441, 119])

In [43]:
# Compute the level-0 (voxel-wise) semantic segmentation predictions 
# based on the predictions on level-1 superpoints and save those for 
# visualization in the level-0 Data under the 'semantic_pred' attribute
nag[0].semantic_pred = output.voxel_semantic_pred(super_index=nag[0].super_index)

In [ ]:
from src.datasets.s3dis import CLASS_NAMES as S3DIS_CLASS_NAMES
from src.datasets.s3dis import CLASS_COLORS as S3DIS_CLASS_COLORS

nag.show(class_names=S3DIS_CLASS_NAMES, class_colors=S3DIS_CLASS_COLORS)